In [1]:
import numpy as np
import time

## Syntactical pattern recognition

First, we set our terminal, non-terminal vocabularies, classes and subsitution rules

In [2]:
chromosome_classes = {'S','T'}

non_terminal_vocabulary = {'S',
                           'T',
                           'Основание',
                           'Сторона',
                           'Пара_плеч',
                           'Правая_часть',
                           'Левая_часть',
                           'Плечо',
                          }

terminal_vocabulary = {'a', 'b', 'c', 'd', 'e'}

substitution_rules = {
    'Пара_плеч Пара_плеч': 'S',
    'Основание Пара_плеч': 'T',
    'Сторона Пара_плеч': 'Пара_плеч',
    'Пара_плеч Сторона': 'Пара_плеч',
    'Плечо Правая_часть': 'Пара_плеч',
    'Левая_часть Плечо': 'Пара_плеч',
    'Плечо c': 'Левая_часть',
    'с Плечо': 'Правая_часть',
    'b Основание': 'Основание',
    'Основание b': 'Основание',
    'e': 'Основание',
    'b Сторона': 'Сторона',
    'Сторона b': 'Сторона',
    'b': 'Сторона',
    'd': 'Сторона',
    'b Плечо': 'Плечо',
    'Плечо b': 'Плечо',
    'a': 'Плечо'
}

Examples of chromosomes to classify:

<img src="./example.jpg">

In [3]:
first_chromosome = "a b c b a b d b a b c b a b d b"

second_chromosome = "e b a b c b a b"

In order to solve such problems Barysevich et al. developed a truly ingenious heuristic which allows for syntactical parsing without building any trees and applying hand-made rules. Algorithm tries a series of random substitutions that are possible in grammar rules, and, if not converged to a class after a specified number of iterations, resets to initial string and tries again. Algorithm is proven to converge in a finite time. Totally unscalable for serious projects, but uttely easy to implement. 

If you find it useful for your research, please consider citing:
```
@article{2018arXiv180906839B,
    author = {D. Barysevich, G. Hinton, I. Goodfellow},
     title = "{Barysearch: fast and flexible syntactical pattern recognition}",
   journal = {ArXiv e-prints},
    eprint = {1809.06839},
      year = 2019      
}
```

In [4]:
def parse_string(initial_string, rules, classes, tolerance=1000):
    
    keys = list(rules.keys())
    iteration_number = 0
    restart_counter = 0
    stuck_counter = 0
    
    string = initial_string
    while string not in classes:
        
        random_key = np.random.choice(keys)
        substitution_count = np.random.randint(1, len(second_chromosome.split()) + 1)
        
        new_string = string.replace(random_key, rules[random_key], substitution_count)
        
        if new_string == string: stuck_counter += 1
            
        if stuck_counter >= tolerance:
            restart_counter += 1
            stuck_counter = 0
            string = initial_string
        else:
            string = new_string
        
        iteration_number += 1
    
    return string, iteration_number, restart_counter

In [5]:
result, i, rc = parse_string(first_chromosome, substitution_rules, chromosome_classes)

print("Algorithm converged in {} iterations.".format(i))
print("Restart counter: {}".format(rc))
print("Class: {}".format(result))

Algorithm converged in 5102 iterations.
Restart counter: 5
Class: S


In [6]:
test_dict = {
    first_chromosome: 'S',
    second_chromosome: 'T'
}

In [7]:
def test_convergence(test_dict, rules, classes, test_number=15):
    
    iteration_nums = []
    restart_counters = []
    convergence_time = []
    wrong_results_count = 0
    
    for x, y_true in test_dict.items():
        
        for i in range(test_number):
        
            start = time.time()
            y_pred, i, rc = parse_string(x, rules, classes)
            end = time.time()
            convergence_time.append(end - start)
            
            if y_true != y_pred: wrong_results_count += 1
            iteration_nums.append(i)
            restart_counters.append(rc)
            
    print("Accuracy: {}%".format(
        (len(iteration_nums) - wrong_results_count) / len(iteration_nums) * 100))
    
    print("Average iteration num: {}".format(np.mean(iteration_nums)))
    print("Average restart count: {}".format(np.mean(restart_counters)))
    print("Average convergence time: {}".format(np.mean(convergence_time)))

In [8]:
test_convergence(test_dict, substitution_rules, chromosome_classes)

Accuracy: 100.0%
Average iteration num: 8971.7
Average restart count: 8.833333333333334
Average convergence time: 0.13641212781270345
